# Die Webseite der TH Nürnberg Scrapen

### Einleitung

Die Webseite der TH-Nürnberg wirkt als Ausgangspunkt für die Wissensgrundlage des Chatbots.

Conventions: 
- pandas Spaltennamen im Sigular
- Die meißten Links sind keine URLs, da sie lokal sind

In [ ]:
import sys
sys.path.append('..')
from bs4 import BeautifulSoup
from bs4.element import Comment
import pandas as pd
from db_init import db_get_df, db_save_df
import glob
import json
import requests
import sys
from tqdm import tqdm

### Scrapen der home Seite

Als Ausganspunkt für unsere Daten nutzen wir die Home Webseite der TH-Nürnberg. (https://www.th-nuernberg.de/)
Diese Website downloaden wir und suchen alle Links auf andere Webseiten und speichern diese Links in eine Liste.
Als nächsten Schritt rufen wir alle Links aus dieser Liste Auf und sammeln wiederum alle Links von jeder dieser Webseiten.
In der daraus resultierenden Liste sortieren wir alle Links aus, die nicht auf die Webseite der TH verweisen.
Dann laden wir alle Dokumente herunter und speichern sie in der Datenbank.

Eine Funktion, die eine URL als parameter nimmt und das HTML file zurückgibt, falls die Seite existiert

In [ ]:
def download_html_from_url(url):
    res = requests.get(url)
    html = ""
    if res.ok:
        html =  res.text
    return html

Eine Funktion, die ein HTML file nach links durchsucht und alle gefundenen externen und internen Links zurückgibt.

In [ ]:
def get_links_from_one_html(html):
    soup = BeautifulSoup(html,"lxml")
    urls = [a["href"] for a in soup.find_all('a', href=True)]
    return urls

Jetzt geben wir unsere initiale URL an und extrahieren alle Links aus dieser

In [ ]:
BASE_URL = "https://www.th-nuernberg.de/"
html = download_html_from_url(BASE_URL)
links = get_links_from_one_html(html)
df = pd.DataFrame({"link": links})
print(*df["link"])

### Links filtern

Zunächst können wir alle Links überprüfen, ob sie Parameter oder sections mit 
- *?param1=hallo*    
- *#section*        

Beide Attribute sind für den Download der Webseiten nicht notwendig und erschweren das finden von Dublikaten also schneiden wir sie hinten ab

In [ ]:
def 

### Externe Links filtern

Jetzt können wir Mal nachschauen, auf welche externen Seiten die THN Webseite verweist.

In [ ]:
def filter_extern_urls(urls):
    filtered_links = []
    for link in urls:
        if link.startswith("http"):
            filtered_links.append(link)

    return filtered_links

In [ ]:
extern_links = filter_extern_urls(df["link"])
print("Anzahl externer Links: ", len(extern_links))
print(extern_links)

Die Seite verweist also auf alle gängigen Sozial Media Seiten, wie Twitter, youtube, tiktok (der nicht existiert), instagram (der nicht existiert), xing, oft auf die jobbörse mit mehreren Links ins Intranet und auf die Efi fakultät,

### Interne Links filtern

Wir filtern nun noch alle Links heraus, die keine Text-inhalte besitzen oder auf die Selbe Seite referieren.

In [ ]:
def filter_intern_links(urls):
    filtered_links = []
    for url in urls:
        if url.startswith("#"):
            continue
        elif url.startswith("http"):
            continue
        elif url.startswith("mailto:"):
            continue
        elif url.startswith("javascript:"):
            continue
        elif url.startswith("&#"): # is encoded mailto
            continue
        elif ".xml" in url:
            continue
        elif ".pdf" in url:
            continue
        elif url == "/":
            continue
        else:
            filtered_links.append(url)
    return filtered_links

In [ ]:
intern_links = filter_intern_links(df["link"])
print("Anzahl interner Links: ", len(intern_links))
print(intern_links)

### Dublikate entfernen und sortieren

Jetzt können wir die Links filtern, anschließend alphabetisch sortieren und dann Dublikate URLs entfernen.

In [ ]:
def sort_and_remove_dublicates(df):
    df = df.sort_values("link")
    df = df.drop_duplicates(subset="link")
    df = df.reset_index(drop=True)
    return df

In [ ]:
intern_links = filter_intern_links(df["link"])
df = pd.DataFrame({"link": intern_links})
df = sort_and_remove_dublicates(df)
print("Anzahl interner Links (ohne Dublikate): ", len(intern_links))
print(*df["link"])

### Weitere Seiten nach urls scrapen

Wir habe also 229 Seiten., die wir jetzt alle nochmal nach links durchforsten können.

In [ ]:
for url in tqdm(df["url"]):
    new_urls = get_links_from_one_url("https://www.th-nuernberg.de" + url)
    new_rows = []
    for new_url in new_urls:
        new_rows.append({'url': new_url})
    df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    

Jetzt können wir wieder schauen, wie viele Interne und externe URLs wir bekommen haben

In [ ]:
print("Anzahl ungefilterter URLs", len(df))
df = sort_and_remove_dublicates(df)
print("Anzahl gefilterter URLs", len(df))
extern_links = filter_extern_urls(df["url"])
print("Anzahl externer URLs", len(extern_links))
intern_links = filter_intern_links(df["url"])
print("Anzahl interner URLs", len(intern_links))

### Links abspeichern

Für unsere weiteren Schritte werden wir immer nur interne Links verwenden, deshalb speichern wir an dieser Stelle mal die internen Links ab.

In [ ]:
db_save_df(df, "only_links")

### Downloaden der files

Jetzt können wir mit dem downloaden anfangen.

Diese Funktion Läd nun alle Html files zu den Links herunter und speichert sie im Dataframe neben den "link" in einer Spalte "html".

In [ ]:
def download_all_urls(links):
    htmls = []
    for link in tqdm(links):
        url = "https://www.th-nuernberg.de" + link
        html = download_html_from_url(url)
        htmls.append(html)
    df = pd.DataFrame({"link": links, "html": htmls})
    return df

In [ ]:
df_new = download_all_urls(df["link"])

Wir können die Daten an dieser Stelle abspeichern.

In [ ]:
db_save_df(df_new, "html_iter_01")

### Weitere Iterationsstufen

Wenn wir ab diesem Abschnitt starten können wir die vorher gesammelten Daten neu laden.

In [ ]:
df = db_get_df("html_iter_01")

Jetzt können wir die heruntergeladenen HTML files nach weiteren Links durchsuchen und Sie dem Dataframe hinzufügen

In [ ]:
def find_all_links_in_html(htmls):
    all_links = []
    for html in tqdm(htmls):
        links = get_links_from_one_html(html)
        [all_links.append(link) for link in links]
    return all_links

In [ ]:
all_links = find_all_links_in_html(df["html"])
len(all_links)

Wir haben jetzt also 71331 Links gesammelt, davon sind aber viele Dublikate.

In [ ]:
df_new = pd.DataFrame({"link": all_links, "html": None})
df = pd.concat([df, df_new])

In [ ]:
df = sort_and_remove_dublicates(df)
len(df["link"])

Gefiltert nach dublikaten haben wir nun also noch 4043 Links

### Iteratives Downloaden

Um den Daten nun weitere Webseiten hinzuzufügen, können wir für jede weitere URL schauen, ob sie schon heruntergeladen wurde. Wenn nicht, dann laden wir sie jetzt herunter.

In [ ]:
def update_df_with_html(df):
    new_rows = []
    for index, row in tqdm(df.iterrows()):
        if pd.isna(row['html']) or row['html'] == '':
            # Call the download function with the link from the 'link' column
            url = "https://www.th-nuernberg.de" + row["link"]
            html = download_html_from_url(url)
            new_rows.append({'link': row["link"], 'html': html})

    if new_rows:
        df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    return df

In [ ]:
df = update_df_with_html(df)

In [ ]:
len(df_ultimate)

In [ ]:
db_save_df(df_ultimate, "html_ultimate")

### Texte extrahieren

Als nächstes müssen wir aus den rohen html dokumenten die unrelevanten Daten aussortieren

In [ ]:
df = db_get_df("html_ultimate")

Die nachfolgende Funktion bestimmt, ob ein Beautifulsoup geparsetes html Element sichtbar ist oder nicht.

In [ ]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

Jetzt bestimmen wir eine Funktion, die ein HTML als Input bekommt und daraus nur die sichtbaren Zeichen bestimmt.

In [ ]:
def get_content(file):
    soup = BeautifulSoup(file,"lxml")

    title = soup.find("title")
    if title:
        title = title.text
    else:
        title = ""

    main = soup.find("main")
    portal = soup.find("div", {'class': 'portal'})
    page_container = soup.find("div", {'class': 'page-wrap'})
    
    visible_texts = ""
    if main:
        # print("main found")
        container = main.find("div" ,{'class': 'container'}, recursive=False)
        if container:
            texts = container.find_all(text=True)
            visible_texts = filter(tag_visible, texts)
        # else:
        #     print("no container") 

    elif portal:
        # print(f"portal found {filename}")
        texts = portal.find_all(text=True)
        visible_texts = filter(tag_visible, texts)
        # print(u" ".join(t.strip() for t in visible_texts))

    elif page_container:
        # print(f"page_container found {filename}")
        container = page_container.find("div" ,{'class': 'container'}, recursive=False)
        # container = page_container.children[2]
        if container:
            texts = container.find_all(text=True)
            visible_texts = filter(tag_visible, texts)


    return {
        "title":    title,
        "text":     u" ".join(t.strip() for t in visible_texts)
    }

In [ ]:
parsed_texts = []
titles = []
for html in tqdm(df["html"]):
    content = get_content(html)
    parsed_texts.append(content["text"])
    titles.append(content["title"])

df["text"] = parsed_texts
df["title"] = titles

In [ ]:

print(df[df["text"] != ""]["url"])